In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import os
data_train = pd.read_csv('../input/mini-flight-delay-prediction/flight_delays_train.csv')
data_test = pd.read_csv('../input/mini-flight-delay-prediction/flight_delays_test.csv')
print(data_train)
data_train.info()

## **Tratando os dados**

In [ ]:
def removeC(data):
  return data[2:]

data_train = data_train.drop(columns = ['UniqueCarrier', 'Origin', 'Dest'])
data_test = data_test.drop(columns = ['UniqueCarrier', 'Origin', 'Dest'])

data_train['dep_delayed_15min'].replace({"Y": 1, "N": 0}, inplace=True)

data_train['Month'] = data_train.apply(lambda row : removeC(row['Month']), axis=1)
data_train['DayofMonth'] = data_train.apply(lambda row : removeC(row['DayofMonth']), axis=1)
data_train['DayOfWeek'] = data_train.apply(lambda row : removeC(row['DayOfWeek']), axis=1)

data_test['Month'] = data_test.apply(lambda row : removeC(row['Month']), axis=1)
data_test['DayofMonth'] = data_test.apply(lambda row : removeC(row['DayofMonth']), axis=1)
data_test['DayOfWeek'] = data_test.apply(lambda row : removeC(row['DayOfWeek']), axis=1)

data_train

In [ ]:
data_test

## **Criando classificador**

In [ ]:
reg = GradientBoostingClassifier(random_state=0, n_estimators=350, learning_rate=0.16)

X_train, y_train = data_train.drop(columns=['dep_delayed_15min']).values, data_train.dep_delayed_15min.values
X_test = data_test.values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=123)

reg.fit(X_train, y_train)
y_pred = reg.predict(X_valid)
proba = reg.predict_proba(X_valid)[:,1]

## **Utilize o roc_auc_score**

In [ ]:
roc_auc_score(y_valid, proba)
pd.Series(proba, 
          name='dep_delayed_15min').to_csv('davidejulioGBDT.csv', 
                                           index_label='id', 
                                           header=True)
print(roc_auc_score(y_valid, proba))